In [ ]:
from IPython.core.display import HTML

with open('style.html', 'r') as file:
    css = file.read()
HTML(css)

# Einleitung

Bei Schach handelt es sich um ein Brettspiel, das insgesamt von zwei Spielern gespielt werden kann.
Ein Spieler hat die Partie gewonnen, sobald er den gegnerischen Spieler Schachmatt gesetzt hat.
Dies ist der Fall, sobald der gegnerische Gegner bedroht wird und der Bedrohung nicht mehr entkommen kann.

## Aufgabenstellung

Das Ziel dieser Ausarbeitung liegt in der Entwicklung einer Schach Endspiel KI.
Mit dieser wird es dem Nutzer ermöglicht für ausgewählte Spielsituationen die Partie optimal zu gewinnen.
Die Berechnung erfolgt mithilfe einer sogenannten Rückwärtsanalyse (engl.: Retrograde Analysis).
Am Ende der Ausarbeitung wird eine KI zur Verfügung gestellt, die für folgende Endspielsituationen eine optimale Spielstrategie beherrscht:

1. König und Dame vs. König
2. König und Turm vs. König
3. König und zwei Läufer vs. König
4. König, Läufer und Springer vs. König
5. König und Bauer vs. König

### Beschreibung der Retrograde Analysis

Die Umsetzung wird sich im Kern auf die Retrograde Analysis fokussieren.
Aus diesem Grund wird diese Methode in Form von diesem Abschnitt kurz erläutert:

1. Zu Beginn werden alle zulässigen Stellungen für eine ausgewählte Spielsituation berechnet. Diese Menge an Stellungen
   werden als Menge $S$ bezeichnet. Hierbei handelt es sich dann um eine zulässige Stellung, wenn der König des Spielers,
   der nicht am Zug ist, im Schach steht.
2. Als Nächstes werden alle Spielsituationen herausgesucht aus der Menge $S$ herausgesucht, bei der der Spieler, der am
   Zug ist, schachmatt gestellt wurde. Für den Rahmen dieser Arbeit sind demnach nur die Stellungen interessant, bei
   denen der gegnerische König schachmatt ist. Diese Stellungen werden in der Menge $S_0$ zusammengefasst und aus der
   Menge $S$ entfernt.
3. Der nächste Schritt bestimmt alle Stellungen aus $S$, in denen der Spieler den gegnerischen Spieler schachmatt setzen
   kann. Das bedeutet, dass von dieser Spielsituation aus mit einem Zug eine Stellung aus der Menge $S_0$ erreicht werden
   kann. Diese Situationen bezeichnen wir als $S_1$. Die Menge $S_1$ wird ebenfalls aus der Menge $S$ entfernt.
4. Danach werden alle Spielsituationen berechnet, in denen der Spieler, der am Zug ist, mit seinem nächsten Zug in der
   Menge $S_1$ landet, also er selbst schachmatt gesetzt werden kann. Diese Stellungen werden als die Menge $S_2$ bezeichnet
   und aus der Menge S herausgenommen.
5. Durch die andauernde Wiederholung des Schrittes 4. wird eine Folge von Mengen $S_n$ erstellt, für die folgende
   Regelungen gelten:
    * Für Stellungen aus den Mengen $S_{2n}$ gilt:
      Diese Menge enthält alle Stellungen, in denen der Spieler, der gerade am Zug ist, nach spätestens $n$ Zügen verloren
      hat. In diesem Fall würden hier alle Stellungen landen, bei denen der König am Zug ist. Dabei gilt zusätzlich,
      dass nach jedem Zug aus der Stellung S_2n in einer Stellung $S_{2k+1}$ landet, wobei $k < n$ gilt.
    * Für Stellungen aus den Mengen $S_{2n+1}$ gilt:
      In dieser Menge sind alle Spielsituationen enthalten in denen der Spieler, der am Zug ist, mit einer optimalen
      Spielweise nach n Zügen gewinnt. Bei den Szenarien handelt es sich hierbei um die Farbe, die mehr als nur einen
      König mehr auf dem Spielfeld stehen haben. Bei diesen Stellungen führt mindestens ein Zug in eine Spielsituation
      $S_{2n}$.

   Die Spielsituationen, die in den Mengen $S_(2n)$ und $S_(2n+1)$ landen, werden aus der Menge $S$ entfernt.
6. Das Verfahren wird nach $n+1$ Schritten beendet, sobald die Menge $S_n$ leer ist. Bei den übrigen Stellungen, die in der
   Menge $S$ verblieben sind, handelt es sich um Spielsituationen, in denen keiner der Spieler einen Sieg erzwingen kann.

Bevor nun in die Umsetzung der Aufgabe eingestiegen wird, werden einige Grundlagen des Schachspiels für den ungeschulten Spieler erklärt. Dadurch können die Schritte zur Berechnung eines optimalen Endspiels besser nachvollzogen werden.

## Schach

Im folgenden Abschnitt soll eine Einführung in den Spielablauf gegeben werden. Diese wird benötigt, um die im Verlauf dieses Dokuments vorgestellten Konzepte zu verstehen.

## Spielbrett und Spielfiguren
Ein Schachbrett besteht aus insgesamt 64 Feldern, die in einer 8x8 Matrix angeordnet sind. Dabei werden die Spalten mit
den Buchstaben a-h und die Zeilen mit den Zahlen 1-8 beschriftet. Das typische Schachmuster entsteht durch einen regelmäßigen Wechsel
zwischen weißen und schwarzen bzw. dunklen und hellen Feldern. Dementsprechend sind insgesamt 32 weiße und 32 schwarze Felder
auf dem Schachbrett zu finden. Alle Felder können mithilfe der zuvor genannten Beschriftung eindeutig identifiziert
werden. Wenn in diesem Dokument Felder durch ein Kürzel wie beispielsweise "c5" spezifiziert werden, ist das Feld
gemeint, das in der Spalte "*Buchstabe*" und der Zeile "*Zahl*" zu finden ist. Das vorherige Beispiel befindet sich also in der dritten (c) Spalte und der fünften (5) Zeile.

Im Folgenden werden die Spalten des Schachbretts als Linien und die Zeilen als Reihen bezeichnet.

Dieses Schachbrett wird maximal von 32 Figuren besetzt (16 weiße und 16 schwarze Figuren). Eine Seite besteht aus
insgesamt sechs unterschiedlichen Figuren. Bei diesen Figuren handelt es sich um:

- Bauern (8)
- Springer (2)
- Läufer (2)
- Türme (2)
- Dame (1)
- König (1)

Die Zahl in den Klammern steht hierbei für die Anzahl an Figuren pro Spieler. Die Aufstellung zu Beginn des Spiels kann
aus der folgenden Abbildung entnommen werden:

![Grundaufstellung](Images/Starting_Position.png)

### Bewegungsmöglichkeiten der Figuren  
Jede der bereits genannten Figuren hat einen Wert und ein Bewegungsmuster. 
Figuren können im Rahmen dieses Bewegungsmuster bewegt werden, werden aber durch andere Figuren blockiert. Eine Figur kann nur 
in Ausnahmefällen übersprungen werden und blockiert grundsätzlich die Bewegungen aller anderen Figuren. 
Figuren des Gegners (der anderen Farbe) können geschlagen werden, indem eine eigene Figur auf dasselbe Feld gestellt wird. 
Eine geschlagene Figur wird vom Spielfeld entfernt. Zwei Figuren derselben Farbe können nicht auf demselben Feld platziert werden.
Die Figuren mit deren Wertigkeit lauten wie folgt (Jonathan Carlstedt, Die kleine Schachschule (2015): S.10ff., S.40):
- **König** (unendlich): Der König gehört zu den unbeweglichsten Figuren auf dem Spielfeld. Er kann pro Zug nur ein Feld entlang
einer Reihe, Linie oder Diagonalen bewegt werden. Dadurch besitzt er jedoch die Möglichkeit, in jede Richtung eine gegnerische Figur
zu schlagen. Als weitere Einschränkung muss beim Ziehen mit dem König beachtet werden, dass das angestrebte Feld nicht durch eine
gegnerische Figur abgedeckt wird. Ein Feld gilt als abgedeckt, wenn eine gegnerische Figur es in einem Zug betreten und die darauf 
stehende Figur schlagen kann. Ist dies der Fall, darf der König nicht auf dieses Feld gesetzt werden.
![König-Bewegungsmuster](Images/King.png)
- **Turm** (5): Der Turm besitzt die Möglichkeit, in einer Reihe oder Linie beliebig viele Felder zu überqueren
(maximal bis zum Ende des Spielfeldes). 
![Turm-Bewegungsmuster](Images/Rook.png)
- **Läufer** (3): Der Läufer kann wie ein Turm in geraden Linien bewegt werden. Er unterscheidet sich dadurch,
dass er nur diagonal bewegt werden kann. Ein Läufer auf dem Feld "a1" kann folglich nur nach "b2" oder entlang der Diagonale
bewegt werden.
![Läufer-Bewegungsmuster](Images/Bishop.png)
- **Dame** (9): Die Dame zählt zu den beweglichsten Figuren auf dem Spielfeld. Sie kombiniert die Bewegungsmuster des
Läufers und des Turms. Das bedeutet, dass sie horizontal (entlang der Reihen), vertikal (entlang der Linien) und diagonal bewegt werden kann.
![Dame-Bewegungsmuster](Images/Queen.png)
- **Bauer** (1): Der Bauer ist die Figur mit der geringsten Beweglichkeit. Dieser kann nur entlang der Linie nach vorne bewegt werden.
Die erste Bewegung jedes Bauern kann ein oder zwei Felder weit sein, folgende Bewegungen sind immer genau ein Feld weit. 
Eine Besonderheit des Bauers liegt in der Richtung, in die ein Bauer gegnerische Figuren schlagen darf.
Dieser darf nur diagonal nach vorne schlagen. Weiter wird der Bauer in eine
beliebige Spielfigur (außer einem Bauern und einem zweiten König) gewandelt, sobald er die Grundlinie des Gegners erreicht hat.
![Bauer-Bewegungsmuster](Images/Pawn.png)
- **Springer** (3): Der Springer besitzt im Gegensatz zu allen bereits beschriebenen Figuren keine lineare Bewegungsrichtung.
Er kann um zwei Felder nach vorne und ein Feld zur Seite versetzt werden. Dieses Verfahren gilt in jede Richtung, 
sodass der Springer im Optimalfall acht Felder erreichen kann. Der Name des Springers kommt dadurch
zustande, dass er die einzige Figur ist, die andere Figuren überspringen kann. Nur das "Zielfeld" kann durch eine eigene
Figur blockiert werden. 
![Springer-Bewegungsmuster](Images/Knight.png)

## Spielablauf

In einem Spiel ziehen die Spieler immer abwechselnd eine Figur ihrer Farbe. Den ersten Zug hat dabei immer weiß.

Beide Spieler verfolgen während der ganzen Partie das Ziel, den gegnerischen Spieler Schachmatt zu setzen. Ein Spieler ist Schachmatt, wenn folgende Bedingungen erfüllt sind:
1. Der König wird durch eine gegnerische Figur bedroht.
2. Der König kann dieser Bedrohung nicht ausweichen.

Eine solche Bedrohung liegt vor, wenn der gegnerische Spieler im nächsten Zug den König schlagen kann. Dies kann auf drei unterschiedliche Weisen pariert werden:
1. Der König bewegt sich aus dem "Schach".
2. Der Spieler schlägt die Schach-gebende Figur.
3. Eine Figur stellt sich zwischen die Schach-gebende Figur und den König.

Ein anderer Spielausgang neben dem Schach-Matt liegt in dem Patt. Ein Patt ist dann gegeben, wenn der Spieler, der am Zug ist, keine Figur mehr ziehen kann und der König des Ziehenden nicht im Schach steht.

Da diese Studienarbeit nicht vorsieht, das komplette Schachspiel zu erklären, werden die restlichen Spielregeln nicht näher erläutert. 
Diese werden aber in der [python-chess](https://python-chess.readthedocs.io/en/latest/) Bibliothek, welche für die Abbildung des Schachspiels im Code verwendet wird, umgesetzt und berücksichtigt.

Anhand der Nummerierung der Notebooks können nun die einzelnen Schritte zur Erstellung, Nutzung und Validierung einer Schach-Endspiel-KI nachvollzogen werden. Die hierfür notwendigen Notebooks sind:

1. `02_calculation_backend.ipynb` (Nachvollziehen der Rechnung)
2. `03_calculation_frontend.ipynb` (Erstellen der Endspieldaten)
3. `04_play_against_ai.ipynb` (Nutzen der Endspieldaten)
4. `05_play_from_history.ipynb` (Wiedergeben eines in 04 gespielten Spiels)
5. `06_stockfish_compare.ipynb` (Vergleich der Endspieldaten mit der Stockfish-Engine)
6. `07_validate_sequences.ipynb` (Überprüfen der berechneten Daten)

Weiter wurden Notebooks, mit unterstützendem Code, der an mehreren Stellen verwendet wird angelegt. Diese haben die Nummerierung 1X. Es handelt sich um die Notebooks:

1. `11_imports.ipynb`
2. `12_integer_management.ipynb` (Integer-Darstellung von Schach-Situationen)
3. `13_mirroring.ipynb` (Spiegeln von Situationen)
4. `14_functions.ipynb` (Allgemeine, häufiger benötigte Funktionen)

Empfehlung zur Lesereihenfolge:

* Notebooks mit Benennung 0X: In Reihenfolge der Nummerierung.
* Notebooks `12_integer_management.ipynb` und `13_mirroring.ipynb`: Wenn Sie in `02_calculation_backend.ipynb` erwähnt werden.
* Notebooks `11_imports.ipynb` und `14_functions.ipynb`: Bei Bedarf für das Verständnis der Funktionen.

# Einrichten der Entwicklungsumgebung

Als ersten Schritt gilt es dieses Github-Repository zu klonen. Dieses ist unter [https://github.com/lukas-becker/chess_endgame_ai.git](https://github.com/lukas-becker/chess_endgame_ai.git) zu finden. In diesem Repository gibt es eine requirements.txt mit der eine virtuelle Umgebung eingerichtet werden kann. Hierzu geht man innerhalb der Anaconda Prompt in den geklonten Ordner. Sobald man diesen erreicht hat, kann man mit folgenden Befehlen ein environment mit dem Namen `chess` erstellen:
> ``conda create -n chess``
> ``conda activate chess``
> ``conda install -c anaconda jupyter``
> ``pip install -r requirements.txt``

Sofern die Umgebung bereits erstellt worden ist, kann man diese mit dem Befehl
> ``conda activate``

aktivieren. Daraufhin gilt es Jupyter Notebook zu starten. Dies gelingt mit der Eingabe von
> ``jupyter notebook``

Am Ende der Bearbeitung des Projekts kann die Umgebung durch
> ``conda deactivate``

beendet werden.

Für die vollständige Nutzung des Projekts werden jedoch weitere Ressourcen benötigt, die nicht in dem Umfang des Repositories bereitgestellt werden. Diese müssen in den darauffolgenden Ordnern hinterlegt sein.
Dabei handelt es sich um:
* [Die Stockfish-Engine](https://stockfishchess.org/download/) (Ordner: stockfish)
* [Die Gaviota-Endspieltabellen](https://chess.cygnitec.com/tablebases/gaviota/) (Ordner: gaviota)
* [Die Syzygy-Endspieltabellen](https://syzygy-tables.info/) (Ordner: syzygy)

Diese können eigenhändig mit den bereitgestellten Links heruntergeladen werden.
Jedoch wird in Form eines Google Drives die benötigten Ressourcen als ZIP-Datei bereitgestellt. Diese kann [hier](https://drive.google.com/file/d/1AwMUGTEsFGLw5NCMUP9eLcU7BZlbnhql/view?usp=drivesdk) gedownloaded werden.

Außerdem sollte bei der Einrichtung beachtet werden, dass folgende Ordner existieren (können auch leer sein): `Played_Games`, `S_n_Results`, `Tests`.